<a href="https://colab.research.google.com/github/Saoudyahya/QuoteAIAgent/blob/main/QuoteAIAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
import random
from typing import Dict, List, Optional
from transformers import AutoModelForCausalLM, AutoTokenizer

class QuoteAIAgent:
    def __init__(self):
        """
        Initialize the Quote AI Agent with default parameters
        """
        try:
            # Language Model Configuration
            self.tokenizer = AutoTokenizer.from_pretrained('gpt2-medium')
            self.model = AutoModelForCausalLM.from_pretrained('gpt2-medium')
        except Exception as e:
            print(f"Error loading model: {e}")
            print("Ensure you have transformers library installed and internet connection.")
            raise

        # Agent-specific parameters
        self.temperature = 0.7
        self.max_length = 100

        # Domain-specific prompt templates
        self.prompt_templates = {
            'business': [
                "Generate a professional quote about {topic} that highlights strategic insights.",
                "Craft a business-oriented quote that captures the essence of {topic}."
            ],
            'motivational': [
                "Create an inspiring quote about {topic} that motivates and empowers.",
                "Develop a thought-provoking motivational quote centered on {topic}."
            ],
            'technical': [
                "Compose a technical quote about {topic} that demonstrates expert knowledge.",
                "Generate an insightful quote that explains the complexity of {topic}."
            ]
        }

        # Context-aware parameters
        self.domain_weights = {
            'business': 0.4,
            'motivational': 0.3,
            'technical': 0.3
        }

        # Quality control parameters
        self.quality_thresholds = {
            'unique_word_ratio': 0.6,
            'min_words': 5,
            'max_words': 30
        }

    def generate_quote(
        self,
        topic: str,
        domain: Optional[str] = None,
        num_candidates: int = 3
    ) -> List[str]:
        """
        Generate multiple quote candidates for a given topic

        Args:
            topic (str): Subject of the quote
            domain (Optional[str]): Specific domain for quote generation
            num_candidates (int): Number of quote candidates to generate

        Returns:
            List[str]: Generated quote candidates
        """
        if not domain:
            domain = self._select_domain()

        prompt_template = random.choice(self.prompt_templates[domain])
        full_prompt = prompt_template.format(topic=topic)

        candidates = []
        attempts = 0
        while len(candidates) < num_candidates and attempts < 10:
            quote = self._generate_single_quote(full_prompt)
            if self._validate_quote(quote):
                candidates.append(quote)
            attempts += 1

        return candidates

    def _select_domain(self) -> str:
        """
        Intelligently select a domain based on predefined weights

        Returns:
            str: Selected domain
        """
        return random.choices(
            list(self.domain_weights.keys()),
            weights=list(self.domain_weights.values())
        )[0]

    def _generate_single_quote(self, prompt: str) -> str:
        """
        Generate a single quote using the language model

        Args:
            prompt (str): Input prompt for quote generation

        Returns:
            str: Generated quote
        """
        input_ids = self.tokenizer.encode(prompt, return_tensors='pt')

        output = self.model.generate(
            input_ids,
            max_length=self.max_length,
            num_return_sequences=1,
            temperature=self.temperature,
            no_repeat_ngram_size=2
        )

        return self.tokenizer.decode(output[0], skip_special_tokens=True)

    def _validate_quote(self, quote: str) -> bool:
        """
        Validate generated quote based on quality metrics

        Args:
            quote (str): Generated quote to validate

        Returns:
            bool: Whether quote meets quality thresholds
        """
        words = quote.split()
        unique_words = len(set(words)) / len(words)

        return (
            unique_words >= self.quality_thresholds['unique_word_ratio'] and
            self.quality_thresholds['min_words'] <= len(words) <= self.quality_thresholds['max_words']
        )

    def update_domain_weights(self, feedback: Dict[str, float]):
        """
        Update domain weights based on user feedback

        Args:
            feedback (Dict[str, float]): Domain performance scores
        """
        total_score = sum(feedback.values())
        self.domain_weights = {
            domain: score / total_score
            for domain, score in feedback.items()
        }

# Example usage
def main():
    try:
        quote_agent = QuoteAIAgent()

        # Generate quotes about innovation
        innovation_quotes = quote_agent.generate_quote(
            topic="innovation",
            domain="business",
            num_candidates=3
        )

        for idx, quote in enumerate(innovation_quotes, 1):
            print(f"Quote {idx}: {quote}")

        # Simulate user feedback
        quote_agent.update_domain_weights({
            'business': 0.8,
            'motivational': 0.6,
            'technical': 0.4
        })

    except Exception as e:
        print(f"An error occurred: {e}")
        print("Ensure you have the 'transformers' library installed.")
        print("You can install it using: pip install transformers torch")

if __name__ == "__main__":
    main()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/26.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/718 [00:00<?, ?B/s]

vocab.json:   0%|          | 0.00/1.04M [00:00<?, ?B/s]

merges.txt:   0%|          | 0.00/456k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/1.36M [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/1.52G [00:00<?, ?B/s]

generation_config.json:   0%|          | 0.00/124 [00:00<?, ?B/s]

/usr/local/lib/python3.10/dist-packages/transformers/generation/configuration_utils.py:590: UserWarning: `do_sample` is set to `False`. However, `temperature` is set to `0.7` -- this flag is only used in sample-based generation modes. You should set `do_sample=True` or unset `temperature`.
  warnings.warn(
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask is not set and cannot be inferred from input because pad token is same as eos token. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_tok

Quote 1: Generate a professional quote about innovation that highlights strategic insights.

Create a custom quote that is relevant to your company.
Quote 2: Generate a professional quote about innovation that highlights strategic insights.

Create a custom quote that is relevant to your company.
Quote 3: Generate a professional quote about innovation that highlights strategic insights.

Create a custom quote that is relevant to your company.


In [4]:

# Create an instance of the Quote AI Agent
quote_agent = QuoteAIAgent()

# Generate quotes about a specific topic
# You can specify a domain or let it choose automatically
quotes = quote_agent.generate_quote(
    topic="innovation",      # The subject of the quotes
    domain="business",       # Optional: specify a domain (business, motivational, technical)
    num_candidates=3         # Number of quote candidates to generate
)

# Print the generated quotes
for idx, quote in enumerate(quotes, 1):
    print(f"Quote {idx}: {quote}")

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.
The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:None for open-end generation.


Quote 1: Generate a professional quote about innovation that highlights strategic insights.

Create a custom quote that is relevant to your company.
Quote 2: Generate a professional quote about innovation that highlights strategic insights.

Create a custom quote that is relevant to your company.
Quote 3: Generate a professional quote about innovation that highlights strategic insights.

Create a custom quote that is relevant to your company.
